In [3]:
import os
from torch.utils.data import DataLoader

data_dir = "../SmallEarthNet/"
cloud_shadow_file = "../patches_with_cloud_and_shadow.csv"
snow_file = "../patches_with_seasonal_snow.csv"
patches = os.listdir(data_dir)
patches.sort()
print(len(patches))

%load_ext autoreload
%autoreload 2

import load_data_tf as load_data

10000
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
data = load_data.BigEarthNetDataset(split_file="splits.pkl")

In [43]:
print(meta_dataset.counts)
print(meta_dataset.dataset.idx_to_label)
sum(meta_dataset.counts.values())

Counter({'Mixed forest': 86198, 'Non-irrigated arable land': 85512, 'Coniferous forest': 81971, 'Broad-leaved forest': 72487, 'Transitional woodland/shrub': 69834, 'Land principally occupied by agriculture, with significant areas of natural vegetation': 69389, 'Complex cultivation patterns': 54184, 'Pastures': 44475, 'Sea and ocean': 37827, 'Discontinuous urban fabric': 31484, 'Water bodies': 29301, 'Agro-forestry areas': 8736, 'Continuous urban fabric': 7682, 'Peatbogs': 7072, 'Natural grassland': 6832, 'Industrial or commercial units': 4982, 'Water courses': 4185, 'Sclerophyllous vegetation': 4150, 'Vineyards': 3603, 'Permanently irrigated land': 3535, 'Olive groves': 3229, 'Inland marshes': 2495, 'Sport and leisure facilities': 2205, 'Annual crops associated with permanent crops': 2096, 'Mineral extraction sites': 1990, 'Bare rock': 1975, 'Fruit trees and berry plantations': 1921, 'Moors and heathland': 1754, 'Road and rail networks and associated land': 1460, 'Rice fields': 940, 'G

738532

In [37]:
meta_dataset = load_data.MetaBigEarthNetTaskDataset(data_dir="../SmallEarthNet", split_save_path="smallearthnet.pkl", split_file="smallearthnet_splits.pkl")

 19%|█▉        | 1859/9607 [00:00<00:00, 18581.69it/s]

Reloading train-val-test split cache from smallearthnet_splits.pkl
File smallearthnet_splits.pkl not found. Creating new split instead with file name smallearthnet.pkl
Building new train-val-test split and saving to smallearthnet.pkl


100%|██████████| 9607/9607 [00:00<00:00, 17363.29it/s]


In [38]:
X, y, yt = meta_dataset.sample_batch(batch_size=4, split='train')
print(y, yt)

[4, 5, 6, 7, 12, 13, 14, 15, 16, 17, 19, 20, 22, 24, 25, 28, 30, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42] 2965
[[[0. 0. 1.]
  [1. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[1. 1. 0.]
  [1. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[0. 1. 0.]
  [0. 0. 1.]
  [0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 0.]
  [0. 0. 1.]
  [1. 0. 0.]
  [1. 1. 0.]]

 [[1. 0. 0.]
  [1. 0. 1.]
  [1. 0. 1.]
  [1. 1. 0.]
  [1. 0. 1.]
  [1. 0. 0.]
  [1. 0. 0.]
  [0. 0. 1.]]] [[[ 7. -1. -1.]
  [ 4.  6. -1.]
  [ 4. -1. -1.]
  [ 7. -1. -1.]
  [ 4.  7. -1.]
  [ 7. -1. -1.]
  [ 7. -1. -1.]
  [ 7. -1. -1.]]

 [[ 4.  7. -1.]
  [ 4. -1. -1.]
  [ 7. -1. -1.]
  [ 7. -1. -1.]
  [14. -1. -1.]
  [ 4. -1. -1.]
  [ 4. -1. -1.]
  [ 4. -1. -1.]]

 [[ 7. -1. -1.]
  [14. -1. -1.]
  [ 7. -1. -1.]
  [ 7. -1. -1.]
  [ 4. -1. -1.]
  [14. -1. -1.]
  [ 4. -1. -1.]
  [ 4.  7. -1.]]

 [[ 4. -1. -1.]
  [ 4.  7. -1.]
  [ 4.  7. -1.]
  [ 4.  5. -1.]
  [ 4.  7. -

In [44]:
train = DataLoader(data, batch_size=16)
img_batch, label_batch = next(iter(train))
img_batch.size(), label_batch.size()

(torch.Size([16, 3, 120, 120]), torch.Size([16, 43]))

In [75]:
train, _, _ = load_data.get_dataloaders(split_file="splits.pkl")

Reloading train-val-test split cache from splits.pkl
Reloading train-val-test split cache from splits.pkl
Reloading train-val-test split cache from splits.pkl


In [76]:
img_batch, label_batch, raw_labels = next(iter(train))
img_batch.size(), label_batch.size()

(torch.Size([8, 8, 3, 120, 120]), torch.Size([8, 8, 3]))

In [77]:
meta_dataset.key_indices, raw_labels

([4,
  5,
  6,
  7,
  12,
  13,
  14,
  15,
  16,
  17,
  19,
  20,
  22,
  24,
  25,
  28,
  30,
  32,
  33,
  34,
  35,
  36,
  37,
  39,
  40,
  41,
  42],
 tensor([[[22, -1, -1],
          [ 4, 17, -1],
          [ 4, -1, -1],
          [ 4, -1, -1],
          [ 4, 22, -1],
          [17, 22, -1],
          [ 4, 22, -1],
          [ 4, 22, -1]],
 
         [[ 4, -1, -1],
          [ 4, 22, -1],
          [ 4, 37, -1],
          [ 4, -1, -1],
          [ 4, 22, -1],
          [ 4, -1, -1],
          [22, -1, -1],
          [ 4, 22, -1]],
 
         [[ 4,  5, 22],
          [ 4,  5, -1],
          [ 4,  5, 22],
          [ 4, -1, -1],
          [ 4, -1, -1],
          [ 4, 22, -1],
          [ 4,  5, 22],
          [22, -1, -1]],
 
         [[ 4, -1, -1],
          [ 7, -1, -1],
          [14, -1, -1],
          [ 4, -1, -1],
          [ 4, 14, -1],
          [ 4, -1, -1],
          [ 4,  7, -1],
          [ 4, 14, -1]],
 
         [[ 4, 22, -1],
          [ 4, 22, -1],
          [ 4

In [ ]:
import csv
import gdal
import rasterio

In [ ]:

elimination_patch_list = []  
for file_path in [cloud_shadow_file, snow_file]:
    if not os.path.exists(file_path):
        print('ERROR: file located at', file_path, 'does not exist')
        exit()
    with open(file_path, 'r') as f:
        csv_reader = csv.reader(f, delimiter=',')
        for row in csv_reader:
            elimination_patch_list.append(row[0])
#print('INFO:', len(elimination_patch_list), 'number of patches will be eliminated')
elimination_patch_list = set(elimination_patch_list)


In [ ]:
filtered_patches = [patch for patch in patches if patch not in elimination_patch_list]
len(filtered_patches)

In [ ]:
os.listdir(os.path.join(data_dir, patches[2]))

In [ ]:
import random
idx = random.randint(0, len(patches))
all_bands = ['B01', 'B02', 'B03', 'B04', 'B05',
              'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
rgb_bands = ['B04', 'B03', 'B02']

band_stack = []
for bands in rgb_bands:
    band_path = os.path.join(data_dir, patches[idx], "{}_{}.tif".format(patches[idx], bands))
    print("Loading", band_path)
    assert os.path.isfile(band_path)

    band_ds = gdal.Open(band_path,  gdal.GA_ReadOnly)
    raster_band = band_ds.GetRasterBand(1)
    band_data = raster_band.ReadAsArray()
    band_stack.append(band_data)



In [ ]:
import matplotlib.pyplot as plt

_OPTICAL_MAX_VALUE = 2000. # magic number from some google guys
img = np.stack(band_stack) / _OPTICAL_MAX_VALUE # (C, W, H)
img = np.clip(img, 0, 1)
plt.imshow(np.transpose(img))


In [ ]:
import json

In [ ]:
with open(os.path.join(data_dir, patches[idx], "{}_labels_metadata.json".format(patches[idx])), 'r') as f:
    metadata = json.load(f)

metadata

In [ ]:
from collections import Counter

c = Counter()
c.update(metadata['labels'])
c

In [ ]:
from collections import Counter
from tqdm.notebook import tqdm
import pickle

label_counts = Counter()
for patch in tqdm(patches):
    with open(os.path.join(data_dir, patch, "{}_labels_metadata.json".format(patch)), 'r') as f:
        metadata = json.load(f)
        label_counts.update(metadata['labels'])
print(label_counts)

with open("label_counts_cache.pkl", "wb") as f:
    pickle.dump(label_counts, f)

In [ ]:
random.choice(label_counts.keys(), 2)